In [1]:
import pandas as pd
import numpy as np
import os
import cv2
import subprocess
import imageio
import imageio_ffmpeg as ffmpeg
import pickle
from IPython.display import Video, display, Image

In [2]:
def convert_video_to_h264_via_avi(input_path, temp_avi="temp.avi", output_path="preview_h264.mp4"):
    # 读取输入视频
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        raise IOError(f"无法打开视频文件: {input_path}")
    
    # 获取视频属性
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # 以 AVI 格式保存临时视频，使用 MJPG 编码
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    avi_writer = cv2.VideoWriter(temp_avi, fourcc, fps, (width, height))
    if not avi_writer.isOpened():
        raise IOError(f"无法创建 VideoWriter，请检查编码器或路径: {temp_avi}")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        avi_writer.write(frame)
    
    cap.release()
    avi_writer.release()
    
    # 使用 imageio-ffmpeg 提供的 ffmpeg 二进制文件进行转码
    ffmpeg_exe = ffmpeg.get_ffmpeg_exe()
    cmd = [
        ffmpeg_exe, "-y", "-i", temp_avi,
        "-c:v", "libx264", "-preset", "fast",
        output_path
    ]
    subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    
    # 在 Notebook 中显示生成的 H.264 视频
    display(Video(output_path, embed=True))

In [4]:

dir_name = "rD_baseline"
dir_name = "rD_Trial_2"
dir_wrong_data_info = "/nfs/turbo/coe-mcity/hanhy/LNDE_Results/" + dir_name + "/check/"
dir_scene_videos = "/nfs/turbo/coe-mcity/hanhy/LNDE_Results/" + dir_name + "/scene_videos/"

file_dir = "2636"

data = pickle.load(open(dir_wrong_data_info + file_dir + ".pkl", "rb"))
wrong_info = data["wrong_and_sigma"] #(time, 7)
# crash_step = data["crash_step"]
# print(f"inference_length:{crash_step}")

wrong_time_1 = np.argwhere(wrong_info[:, :, 0] > 0.5)
wrong_time_2 = np.argwhere(wrong_info[:, :, 1] > 0.5)
wrong_time_3 = len(np.argwhere(wrong_info[:, :, 2] > 0.5)) > 0
# wrongpred3 = np.argwhere(wrong_sigma_info3[:, :, 3] > 0.5)
# wrongpred4 = np.argwhere(wrong_sigma_info3[:, :, 4] > 0.5)

print(f"video_len:{wrong_info.shape[1] + 1}")
print("wrong dis")
print(wrong_time_1)
print('wrong angle:')
print(wrong_time_2)
print('if small poc:')
print(wrong_time_3)

# print("----------------")
# print(wrongpred3)
# print("----------------")
# print(wrongpred4)
#show video
save_preview_dir = "/home/hanhy/ondemand/data/sys/myjobs/Conflict_Identifier_Network/priview_output_video/"
video_name = dir_scene_videos + file_dir + ".mp4"
assert os.path.exists(video_name)
convert_video_to_h264_via_avi(input_path=video_name, temp_avi=save_preview_dir + "temp.avi", output_path=save_preview_dir + "preview_h264.mp4")

video_len:46
wrong dis
[]
wrong angle:
[]
if small poc:
False


In [10]:
# from left to right : x   from bottom to up: y 
dir_name = "rD_Trial_2"
file_ori = "/nfs/turbo/coe-mcity/hanhy/LNDE_Results/" + dir_name + "/1/" 
file_t = "/nfs/turbo/coe-mcity/hanhy/LNDE_Results/" + dir_name + "/2/"
file_save = "/nfs/turbo/coe-mcity/hanhy/LNDE_Results/" + dir_name + "/check/"

scenes_all = os.listdir(file_ori)
for scene in scenes_all:
    scene = "2636.pkl"
    test_id = 1
    scene_data_file = pickle.load(open(file_ori+scene, "rb"))
    scene_data = scene_data_file["states_considered"] #attention: the last buff is the crash buff and should not be considered
    #check if the time buff is out of 3 sigma or 2 sigma. 
    #then check if the time buff is "wrong". we want to know if there is out of 3 sigma or 2 sigma before the wrong data.abs
    #save the out of x-sigma and wrong time buff in terms of every scene.
    scene_info = {}
    scene_tb_length = len(scene_data)
    scene_info["tb_len"] = scene_tb_length
    
    #Check out the wrong data
    #first get all car info
    vid_all = []
    for i in range(scene_tb_length):
        for j in range(len(scene_data[i])):
            car_id = int(scene_data[i][j].id)
            if car_id not in vid_all:
                vid_all.append(car_id)
    vid_all = sorted(vid_all)

    scene_info["wrong_and_sigma"] = np.zeros((len(vid_all), scene_tb_length, 7))
    scene_info["distance_info"] = -np.ones((len(vid_all), scene_tb_length, 6))
    scene_info["heading_info"] = -np.ones((len(vid_all), scene_tb_length))

    all_cars_posi = -np.ones((len(vid_all), scene_tb_length, 2))
    for i in range(scene_tb_length):
        for car in scene_data[i]:
            car_id = int(car.id)
            j = vid_all.index(car_id)
            all_cars_posi[j, i, :] = np.array([car.location.x, car.location.y])
            scene_info["heading_info"][j, i] = car.speed_heading

    scene_info["distance_info"][:, :, :2] = all_cars_posi

    #then check if angle or dis is too big
    for i in range(len(vid_all)):
        position = all_cars_posi[i]
        if i == test_id:
            print(position)
            print(scene_info["heading_info"][i,:])
        heading = []
        for j in range(1, len(position)):
            if position[j][0] == -1 or position[j-1][0] == -1:
                continue
            #heading.append([np.arctan2(position[j][1] - position[j - 1][1], position[j][0] - position[j - 1][0]), j])
            dis_car = np.sqrt((position[j][1] - position[j - 1][1])**2 + (position[j][0] - position[j - 1][0])**2)
            heading.append([np.arctan2(position[j][1] - position[j - 1][1], position[j][0] - position[j - 1][0]), j, dis_car])
            if dis_car >= 10:
                scene_info["wrong_and_sigma"][i, j, 0] = 1
            
        if len(heading) < 2:
            continue
        for j in range(len(heading) - 1):
            if i == test_id:
                print(j)
                print(heading[j][1], heading[j + 1][1])
                
                print(heading[j][0]*180/np.pi, heading[j+1][0]*180/np.pi)
                print(scene_info["heading_info"][i,heading[j][1]], scene_info["heading_info"][i,heading[j+1][1]])
                print(abs(scene_info["heading_info"][i,heading[j][1]]-heading[j][0]*180/np.pi), abs(scene_info["heading_info"][i,heading[j+1][1]]-heading[j+1][0]*180/np.pi))
                print(abs(heading[j][0] - heading[j + 1][0])*180/np.pi)
                print(abs(2 * np.pi - abs(heading[j][0] - heading[j + 1][0]))*180/np.pi)
                print(heading[j][2], heading[j+1][2])
            
            if abs(heading[j][0] - heading[j + 1][0]) >= np.pi / 4 or abs(2 * np.pi - abs(heading[j][0] - heading[j + 1][0])) >= np.pi / 4:
                if heading[j][2] > 0.5 a

                continue
            if heading[j][2] < 0.6 or heading[j+1][2] < 0.6:
                continue

            scene_info["wrong_and_sigma"][i, heading[j + 1][1], 1] = 1


    break

[[ 77.9526     -73.6538    ]
 [ 78.0314     -73.6069    ]
 [ 78.082      -73.5637    ]
 [ 78.1169     -73.5337    ]
 [ 78.1277     -73.5134    ]
 [ 78.00190545 -73.50726837]
 [ 77.83798029 -73.57806911]
 [ 77.67118678 -73.75115754]
 [ 77.45585638 -73.92692573]
 [ 77.26406046 -74.16384054]
 [ 76.96076096 -74.44241181]
 [ 76.51116608 -74.80037195]
 [ 76.00940286 -75.30416633]
 [ 75.54410142 -75.89041986]
 [ 75.12780215 -76.56815734]
 [ 74.75842086 -77.20923216]
 [ 74.36002079 -77.76951119]
 [ 73.98283549 -78.22793945]
 [ 73.65615237 -78.56284201]
 [ 73.45170031 -78.74884512]
 [ 73.3520356  -78.79674549]
 [ 73.30412092 -78.75370065]
 [ 73.28434999 -78.65781739]
 [ 73.29378331 -78.52469237]
 [ 73.44285046 -78.2990856 ]
 [ 73.63359136 -77.98895396]
 [ 73.8679783  -77.6073023 ]
 [ 74.15016512 -77.16326126]
 [ 74.44788119 -76.68831216]
 [ 74.77958468 -76.24189407]
 [ 75.09927261 -75.84339727]
 [ 75.44367844 -75.44563838]
 [ 75.67180979 -75.19241912]
 [ 75.62407644 -75.23188718]
 [ 75.36956374

In [7]:
dir_name = "rD_gmm_2"
dir_scene_videos = "/nfs/turbo/coe-mcity/hanhy/LNDE_new_gmm/" + dir_name + "/scene_videos/"

file_dir = "8"

save_preview_dir = "/home/hanhy/ondemand/data/sys/myjobs/Conflict_Identifier_Network/priview_output_video/"
video_name = dir_scene_videos + file_dir + ".mp4"
assert os.path.exists(video_name)
convert_video_to_h264_via_avi(input_path=video_name, temp_avi=save_preview_dir + "temp.avi", output_path=save_preview_dir + "preview_h264.mp4")